# Season 9 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains information that is not currently present on the main site and may be considered spoilers. Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

To run, press the ⏩ button above and select "Run all cells".

Cells can be modified and rerun at any time. Changes will not be saved when the session ends.

In [ ]:
%matplotlib inline
from beakerx import *
from display import *
from blessings import *
from blaseball_mike.models import *

pies = Team.load_by_name("Philly Pies")

# Decrees
All Decrees are “Forecast: X” where X is a pre-existing weather. Each weather has a small chance for specific effects on
the game, for reference:
* Solar Eclipses cause Incinerations.
* Peanuts cause yummy/allergic peanut reactions (±20% to all stats) as well as giant falling peanuts that can shell
  players mid-game.
* Birds cause batters to occasionally ~~get out in the middle of their at-bat~~ (removed due to bugs), and is the only
  known way to break Shelled players free.
* Feedback causes players to swap teams.
* Reverb can either shuffle the lineup, rotation, or whole team order. It can also cause a player to become
  Reverberating, which permanently gives them a chance to bat twice in a row.
* Blooddrain causes players to drain a specific stat (10%) from a player on the other team.

3 Decrees will pass at the end of the election. It is unknown how these decrees will affect the weather but it is
believed that the 3 chosen weathers will be the only weathers available for the next season.

Birds is an obvious benefit for us due to our Affinity for Crows (50% better batting/pitching for the Pies) and is
generally supported by teams that need to break players out of shells. Teams tend to dislike Feedback and Solar Eclipses
as it can kill or trade players they like. Strategically, Peanuts is the worst as it causes a player to lose on average
1 star in each category OR completely remove them from games until they are unshelled.

___
# Blessings

## Eat me, Drink me
**A random player on your team becomes BIG. Increased Power, Max Vibe. Decreased Baserunning.**

~Random~. This is identical to the Mushroom without being an item. Power is a combination of `musclitude` and `divinity`
which would increase triples & home runs. Pies generally have low baserunning and high power already.

## Credit to the Team
**The worst player on your team will earn 5x payouts for Idol Pendants.**

The current worst player on the team is below. Having a player be the target of idol board shenanigans has gone poorly in the
past (see York Silk in a peanut).

In [ ]:
get_stars(sort_overall(pies, 1, order="worst"))

## Secret Weapon
**Maximize a random Pitcher in your Shadows.**

Random. The pitcher Shadows is the bullpen, a list of which can be found below. Just maximizing a player is somewhat
useless unless we also get a blessing to add them to our rotation.

In [ ]:
# Stars
get_stars(pies.bullpen)

In [ ]:
# Pitching stlats
get_pitching_stlats(pies.bullpen)

## Grab and Smash
**Steal a Hitter from another Team's Shadows.**

The hitter Shadows is the bench. This would likely replace our worst batter, found below. Below is a graph of all the
possible batters in division opponent benches sorted by number of stars. Previous seasons have taken the best player
from the Shadows, but that’s a sample size of 1 so it’s hard to tell if it was just luck.

In [ ]:
# Will replace this Pies player:
get_stars(sort_lineup(pies, 1, order="worst"))

In [ ]:
gs_bat, gs_base, gs_def = grab_and_smash(pies)

# Batting Stars
plot = Plot(title="Grab and Smash - Batting", xLabel="Batting Stars", yLabel="Number of Shadows Players")
plot.add(Bars(y=gs_bat["Total"], x=gs_bat.index, width=0.4))

In [ ]:
# Baserunning Stars
plot = Plot(title="Grab and Smash - Baserunning", xLabel="Baserunning Stars", yLabel="Number of Shadows Players")
plot.add(Bars(y=gs_base["Total"], x=gs_base.index, width=0.4))

In [ ]:
# Defense Stars
plot = Plot(title="Grab and Smash - Defense", xLabel="Defense Stars", yLabel="Number of Shadows Players")
plot.add(Bars(y=gs_def["Total"], x=gs_def.index, width=0.4))

## Out of Sight
**Send your 2 Worst Pitchers to the Shadows and call up your First 2 Relievers.**

See below for comparison between existing pitchers and their replacements.

In [ ]:
OOS_old = sort_rotation(pies, 2)
OOS_new = pies.bullpen[0:2]

In [ ]:
# Star Comparison
table = TableDisplay(get_stars(OOS_old + OOS_new))
table = set_heatmap(table, maxVal=5)
table

In [ ]:
# Pitching Stlat Comparison
table = TableDisplay(get_pitching_stlats(OOS_old + OOS_new))
table = set_heatmap(table, maxVal=1)
table

## Disappearing Acts
**Send your Worst 3 Hitters to the Shadows and call up the first 3 Shadows Hitters.**

See below for comparison between existing batters and their replacements.


In [ ]:
DA_old = sort_lineup(pies, 3)
DA_new = pies.bench[0:3]

In [ ]:
# Star Comparison
table = TableDisplay(get_stars(DA_old + DA_new))
table = set_heatmap(table, maxVal=5)
table

In [ ]:
# Batting Stlat Comparison
table = TableDisplay(get_batting_stlats(DA_old + DA_new))
table = set_heatmap(table, maxVal=1)
table

## Feel the Love
**Give your Team Love Blood Type.**

Currently Love Blood does not have an in-game impact. It will probably gain a gameplay mechanic once the blessing is
passed, but this is currently unknown.

## Hot Sauce Bar
**Help yourselves. A player on each team in your Division gets Spicy.**

Spicy gives a player improved batting if they get 3 hits in a row, which lasts until they get out at-bat again
(strikeout, fly or ground out). Pies batting is strong but not generally known for consistency.

## Fifth Base Bubble
**Give all of the teams not in your Division the Fifth Base.**

Exact impact unknown. A Fifth base is generally bad for teams that have weak baserunning and hitting.

## The Shame Bubble
**Give all of the teams not in your Division Targeted Shame.**

We had this in Season 8, not sure how much of an impact it made for us specifically.

## Tag Team Pitching
**Improve your team's pitching by 10%. The last place team in the opposite subleague also gets +10% team pitching.**

See below for analysis.

In [ ]:
# Pies Improvement
table, total, avg = improve_team_pitching_pandas(pies, 0.10)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
table

In [ ]:
# Opponent Improvement
tacos = Team.load_by_name("Tacos")
table, total, avg = improve_team_pitching_pandas(tacos, 0.10)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
table

## Tag Team Hitting
**Improve your team's hitting by 10%. The last place team in the opposite subleague also gets +10% team hitting.**

See below for analysis.

In [ ]:
# Pies Improvement
table, total, avg = improve_team_batting_pandas(pies, 0.10)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
table

In [ ]:
# Opponent Improvement
tacos = Team.load_by_name("Tacos")
table, total, avg = improve_team_batting_pandas(tacos, 0.10)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
table

## Kill Your Darling
**Incinerate the Most Idolized Player on your Team.**

Hey.

Don’t.

## Dead Weight
**Incinerate the Worst Player on Your Team.**

See below for analysis. The sliders will adjust the Batting, Pitching, Baserunning, and Defense stars of the randomly generated player. They start at the statistically average generated player.

In [ ]:
dead_weight_form = player_gen_UI("Dead Weight Replacement Player", tag_name="dead_weight")
dead_weight_form

In [ ]:
# Would replace the following Pie
worst_player = sort_overall(pies, num=1)[0]
get_stars(worst_player)

In [ ]:
# Change in team average stars
pandas.DataFrame(replace_player(pies, worst_player, dead_weight_form["batting"], dead_weight_form["pitching"], dead_weight_form["baserunning"], dead_weight_form["defense"])).transpose()

## New Kid
**Incinerate the Least Idolized Player on your Team.**

Cannot currently know who the least-idolized player on the team is. With correct organization could be used as a
controlled incineration for one of our bad players.

## Boatload of Tiny Eggs
**Fans of your team receive a big barrel of peanuts.**

Peanuts would be used for the Hall of Flame ratings. Amount of peanuts in a “boatload” is unknown; last season’s
blessing was 10K.

## New Recruit
**Recruit a rookie to be an extra player in your lineup.**

This would cause the existing players to bat less, which means we would need to get a reroll of over our team average
to not lose total team stars. See below for analysis. The sliders will adjust the Batting, Pitching, Baserunning, and Defense stars of the randomly generated player. They start at the statistically average generated player.

In [ ]:
new_recruit_form = player_gen_UI("New Recruit Player", tag_name="new_recruit")
new_recruit_form

In [ ]:
# Change in team average stars
pandas.DataFrame(add_player(pies, new_recruit_form["batting"], new_recruit_form["baserunning"], new_recruit_form["defense"])).transpose()

## Downsizing
**Remove the worst player from your lineup. Your lineup is now shorter.**

See below for analysis

In [ ]:
# Would replace the following Pie
worst_lineup = sort_lineup(pies, num=1)[0]
get_stars(worst_lineup)

In [ ]:
# Change in team average stars
pandas.DataFrame(remove_player(pies, worst_lineup)).transpose()

## Fishing Net
**Item. Max Out Defense for a random player on your team.**

Random. Overall Pies defense is decent and improving it for one player will likely not be a large change.

## Mutually Arising
**Improve your entire division by 2%**

See below for analysis

In [ ]:
# Compute team average stars
mild_low = Division.load_by_name("Mild Low")
mutual_arise = DataFrame()
for team in mild_low.teams.values():
    _, _, average = improve_team_overall_pandas(team, 0.02)
    average = average.rename(team.nickname)
    mutual_arise = mutual_arise.append(average)
mutual_arise.transpose()

# Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/FK:Player_Attributes)

